# Understanding the structure of the results from experiments

In [15]:
import pickle
import os
import numpy as np

In [18]:
experiment_path = r"C:\Users\david\Documents\GitHub\ml2\experiments\baseline\break_sequence\equal_balancing_oversampling\label_language\2023_12_18_0"


# Load the variables
with open(os.path.join(experiment_path, "results\\nonnested_cval_mltsatt_l819.pkl"), 'rb') as f:
    results = pickle.load(f)

with open(os.path.join(experiment_path,'config.yaml'), 'rb') as f:
    config_info = pickle.load(f)

In [19]:
config_info

{'experiment': {'root_name': 'baseline/break_sequence/equal_balancing_oversampling/label_language',
  'name': '/2023_12_18_0/',
  'type': 1,
  'labels': 'binconcepts'},
 'data': {'key': 'break_sequence',
  'others': ({'gender': ['3', '4']},),
  'adjuster': {'limit': 819}},
 'ml': {'nfolds': 10,
  'epochs': 30,
  'oversampler': {'oversampling_factor': 1.5,
   'oversampling_col': ['label', 'language'],
   'rebalancing_mode': 'equal_balancing',
   'addyourownparameters': 'helloworld',
   'mode': 'none'},
  'splitter': {'stratifier_col': ['stratifier_column']}},
 'seeds': {'splitter': 219, 'oversampler': 13, 'model': 139, 'numpy': 36},
 'mode': 'baseline'}

## Results Analyse

In [20]:
results.keys()

dict_keys(['x', 'y', 'demographics', 'optim_scoring', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [21]:
len(results["x"]),len(results["y"]) # 100 % of the original data

(254, 254)

In [22]:
print("Demographics keys: ",results["demographics"].keys())
print("optim scoring keys: ",results["optim_scoring"], " ; is a ", type(results["optim_scoring"]))

Demographics keys:  dict_keys(['gender', 'year', 'language', 'label', 'stratifier_col', 'oversampler_col'])
optim scoring keys:  roc  ; is a  <class 'str'>


In [23]:
# Correspond to 100 % of the original data
print(len(results["demographics"]["oversampler_col"]))

# count the number of unique values in the demographics
print(np.unique(results["demographics"]["oversampler_col"], return_counts=True))
print(np.unique(results["demographics"]["label"], return_counts=True))

254
(array(['0Deutsch', '0Français', '1Deutsch', '1Français'], dtype='<U9'), array([96, 47, 82, 29], dtype=int64))
(array(['0', '1'], dtype='<U1'), array([143, 111], dtype=int64))


## Study of 1 fold

In [25]:
# The numbers for 0 to 10 are the results of each of the 10 folds
print(results[1].keys())

dict_keys(['train_index', 'test_index', 'oversample_indexes', 'x_resampled', 'y_resampled', 'x_resampled_train', 'y_resampled_train', 'x_resampled_val', 'y_resampled_val', 'best_params', 'gender', 'year', 'language', 'label', 'y_pred', 'y_proba', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc', 'tp', 'fp', 'fn'])


In [26]:
train_indices = results[1]["train_index"]
demographics_data = results["demographics"]["oversampler_col"]

# Get the demographic information for the training indices
train_demographics = [demographics_data[i] for i in train_indices]

info_demo_train_fold_1 = np.unique(train_demographics, return_counts=True)
print(info_demo_train_fold_1)

(array(['0Deutsch', '0Français', '1Deutsch', '1Français'], dtype='<U9'), array([87, 41, 72, 28], dtype=int64))


In [29]:
print(results[1]["train_index"][0:10], len(results[1]["train_index"]), "\n")
print(results[1]["test_index"], len(results[1]["test_index"]), "\n")
print(len(results[1]["oversample_indexes"]),"; # of unique values:",len(np.unique(np.array(results[1]["oversample_indexes"]))),"\n")

a = info_demo_train_fold_1[1].max()
print("If equal ov:","demo after os:", a*2 , "|lables after os: ",(a*2)*2,"|total data after os:",(a*2)*2 *2)

[ 0  1  3  4  6  7  8  9 10 11] 228 

[  2   5  13  23  40  83 104 112 121 126 142 155 158 161 166 169 174 184
 200 202 214 217 223 224 240 242] 26 

228 ; # of unique values: 228 

If equal ov: demo after os: 174 |lables after os:  348 |total data after os: 696


In [30]:
len(results[1]["x_resampled"]), len(results[1]["x_resampled_train"]), len(results[1]["x_resampled_val"])

(228, 228, 26)

In [31]:
results[1]["x_resampled"] == results[1]["x_resampled_train"]

True

In [32]:
# The data given to the Classifier afther oversampling !
results[1]["x_resampled_train"][0][0], results[1]["x_resampled_train"][-1][0] # fold 1 with the number of student x number of their sequences

([0, 0, 0, 1, 0, 0, 0, 0, 0, 7.889], [0, 0, 0, 1, 0, 0, 0, 0, 0, 5.447])

In [33]:
results[1].keys()

dict_keys(['train_index', 'test_index', 'oversample_indexes', 'x_resampled', 'y_resampled', 'x_resampled_train', 'y_resampled_train', 'x_resampled_val', 'y_resampled_val', 'best_params', 'gender', 'year', 'language', 'label', 'y_pred', 'y_proba', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'roc', 'tp', 'fp', 'fn'])

In [34]:
results[1]["best_params"]

{'seed': 1,
 'batch_size': 16,
 'cell_type': 'GRU',
 'dropout': 0.02,
 'early_stopping': False,
 'loss': 'auc',
 'n_cells': [32],
 'n_layers': 1,
 'optimiser': 'adam',
 'padding_value': -1,
 'shuffle': True,
 'verbose': -1,
 'save_best_model': False,
 'flatten': 'average',
 'epochs': 30}

In [35]:
results[1]["y_pred"][0:2], results[1]["y_proba"][0:2]

([1, 1],
 array([[0.08778068, 0.9122193 ],
        [0.2117777 , 0.78822225]], dtype=float32))

In [200]:
results[1]["accuracy"], results[1]["balanced_accuracy"], results[1]["precision"], results[1]["recall"], results[1]["roc"]

(0.6923076923076923,
 0.6727272727272727,
 0.6666666666666666,
 0.5454545454545454,
 0.6909090909090909)

In [201]:
# Make mean of the results acrross the 10 folds
mean_roc = []
for i in range(10):
    mean_roc.append(results[i]["roc"])
mean_roc = np.array(mean_roc).mean()

print("Mean ROC: ", mean_roc)

Mean ROC:  0.6035389610389611


In [89]:
results[1]["label"]

{'fn': {'0': -1, '1': 0.45454545454545453},
 'tn': {},
 'roc': {'0': -1, '1': -1},
 'recall': {'0': 0.0, '1': 0.5454545454545454},
 'precision': {'0': 0.0, '1': 1.0},
 'balanced_accuracy': {'0': 0.8, '1': 0.5454545454545454}}

In [90]:
results[1]["language"]

{'fn': {'Deutsch': 0.5, 'Français': 0.0},
 'tn': {},
 'roc': {'Deutsch': 0.7000000000000001, 'Français': 0.8333333333333334},
 'recall': {'Deutsch': 0.5, 'Français': 1.0},
 'precision': {'Deutsch': 0.8333333333333334, 'Français': 0.3333333333333333},
 'balanced_accuracy': {'Deutsch': 0.6944444444444444,
  'Français': 0.8333333333333333}}